### PuLP Example
https://memo.barrucadu.co.uk/scheduling-problems.html

In [1]:
import pulp

# params
slots  = 5
people = ["Spongebob", "Squidward", "Mr. Crabs", "Pearl"]
roles  = ["Fry Cook", "Cashier", "Money Fondler"]
leave  = {"Mr. Crabs": [0,2,3,4]}
max_assignments_per_person = 5

# Create the 'problem'
problem = pulp.LpProblem("rota generator", sense=pulp.LpMaximize)

# Create variables
assignments = pulp.LpVariable.dicts("A", ((s, p, r) for s in range(slots) for p in people for r in roles), cat="Binary")
is_assigned = pulp.LpVariable.dicts("X", people, cat="Binary")

# Add constraints
for slot in range(slots):
    for role in roles:
        # In every time slot, each role is assigned to exactly one person
        problem += pulp.lpSum(assignments[slot, person, role] for person in people) == 1

    for person in people:
        # Nobody is assigned multiple roles in the same time slot
        problem += pulp.lpSum(assignments[slot, person, role] for role in roles) <= 1

for person, bad_slots in leave.items():
    for slot in bad_slots:
        for role in roles:
            # Nobody is assigned a role in a slot they are on leave for
            problem += assignments[slot, person, role] == 0

for person in people:
    # Nobody works too many shifts
    problem += pulp.lpSum(assignments[slot, person, role] for slot in range(slots) for role in roles) <= max_assignments_per_person

# Constrain 'is_assigned' auxiliary variable
for slot in range(slots):
    for person in people:
        for role in roles:
            # If
            problem += is_assigned[person] >= assignments[slot, person, role]

for person in people:
    # Only if
    problem += is_assigned[person] <= pulp.lpSum(assignments[slot, person, role] for slot in range(slots) for role in roles)

# Add objective
problem += pulp.lpSum(is_assigned[person] for person in people)

# Solve with the Coin/Cbc solver
problem.solve(pulp.solvers.COIN_CMD())

# Print the solution!
for slot in range(slots):
    print(f"Slot {slot}:")
    for role in roles:
        for person in people:
            if pulp.value(assignments[slot, person, role]) == 1:
                print(f"    {role}: {person}")
                


Slot 0:
    Fry Cook: Pearl
    Cashier: Squidward
    Money Fondler: Spongebob
Slot 1:
    Fry Cook: Spongebob
    Cashier: Mr. Crabs
    Money Fondler: Pearl
Slot 2:
    Fry Cook: Spongebob
    Cashier: Squidward
    Money Fondler: Pearl
Slot 3:
    Fry Cook: Spongebob
    Cashier: Pearl
    Money Fondler: Squidward
Slot 4:
    Fry Cook: Squidward
    Cashier: Spongebob
    Money Fondler: Pearl


### Adapt PuLP example to our problem

In [168]:
from DataLoader import *
from CheckSchedule import checkSchedule
from satUtils import *
from data_cleaning import *
import pulp
import numpy as np

In [185]:
# turn leave from date range into a list of unavailable days
def make_leave_successive(pilots):
    leave = {pilot: details['Leave'] for pilot, details in pilots.items()}
    for p in leave.keys():
        p_leave = []
        for l in leave[p]:
            p_leave += range(int(l[0]), int(l[1]+1))
        leave[p] = p_leave
    return leave

def times_overlap(x1, x2, y1, y2) -> bool:
    return max(x1, y1) <= min(x2, y2)

def is_on_leave(event: dict,
                member: int,
                member_details: dict) -> bool:
    # check if on leave
    if not member_details['Leave']:
        return False
    else:
        for dates in member_details['Leave']:
            leave_start = dates[0]
            leave_end = dates[-1]
            # if no overlap:
            if times_overlap(leave_start, leave_end, event['StartDay'], event['EndDay']):
                return True
    return False

def event_range(event):
    if event['EndDay'] == 'nan' or np.isnan(event['EndDay']):
        event['EndDay'] = event['StartDay']
    return range(int(event['StartDay']), int(event['EndDay']+1))

In [194]:
pilots = loadPilotData()
pilots = clean_invalid_leave(pilots)
pilots = clean_pilot_quals(pilots) 
events = loadEventData()

max_assignments = 100 # large number...

# PuLP LP problem formulation
problem = pulp.LpProblem("rota generator", sense=pulp.LpMaximize)

# create variables
# A = rota assignments matrix
# X = aux variable to maximize in obj function
assignments = pulp.LpVariable.dicts('A',
                                    # params: day, pilot, role, event
                                    ((day, pilot, role, event) for event in events
                                                        for pilot in pilots
                                                        for day in event_range(events[event])
                                                        for role in events[event]['CrewRequirements']),
                                    cat='Binary')

is_assigned = pulp.LpVariable.dicts('X', pilots, cat="Binary")

# Constraints
# TODO: constraint to not assign pilots to overlapping events


# TODO: constrain based on quals


for event in events:
    for day in event_range(events[event]):
        for role in events[event]['CrewRequirements']: 

            # Assumption: for simplicity and getting this example to work:
            # assume each events roles are the CrewRequirements

            # exactly 1 pilot assigned to each role in an event
            problem += pulp.lpSum(assignments[day, pilot, role, event] for pilot in pilots) == 1

        for pilot in pilots:
                # 1 pilot cannot fill multiple roles in an event
                problem += pulp.lpSum(assignments[day, pilot, role, event] for role in events[event]['CrewRequirements']) <= 1
        
for pilot in pilots:
    for event, event_details in events.items():
        for day in event_range(events[event]):
            for role in events[event]['CrewRequirements']:
                if is_on_leave(event_details, pilot, pilots[pilot]):
                    # cannot be assigned to an event when on leave
                    problem += assignments[day, pilot, role, event] == 0
    
for pilot in pilots:
    # no pilot flies more than max_assignments
    problem += pulp.lpSum(assignments[day, pilot, role, event] for event in events 
                                                          for day in event_range(events[event]) 
                                                          for role in events[event]['CrewRequirements']) <= max_assignments

# The code commented out below defines the objective function to maximize
# However, PuLP can run with an arbitraty obj function and the problem will be solved as a pseudo-SAT probelm
# Constrain auxiliary variable
# for event in events:
#     for pilot in pilots:
#         for role in events[event]['CrewRequirements']:
#             problem += is_assigned[pilot] >= assignments[event, pilot, role]

# # for pilot in pilots:
#     problem += is_assigned[pilot] <= pulp.lpSum(assignments[event, pilot, role] for event in events
#                                                                                 for role in events[event]['CrewRequirements'])
    
    
# objective function to maximize
# problem += pulp.lpSum(is_assigned[pilot] for pilot in pilots)
problem += 0 # arbitrary obj function

# solve!
print('Solving!')
problem.solve(pulp.solvers.COIN_CMD())
print('Fin...')

Solving!
Fin...


In [195]:
for event in events:
    print(f"Event {event}:")
    for role in range(len(events[event]['CrewRequirements'])):
        for pilot in pilots:
            for day in event_range(events[event]):
                if pulp.value(assignments[day, pilot, events[event]['CrewRequirements'][role], event]) == 1:
                    print(f"    {role}: {pilot}")

Event f1:
    0: 10049
    0: 10040
    0: 10040
    0: 10016
    0: 10016
    0: 10016
    1: 10058
    1: 10054
    1: 10054
    1: 10057
    1: 10040
    1: 10016
    2: 10054
    2: 10057
    2: 10057
    2: 10040
    2: 10016
    2: 10016
Event f2:
    0: 10016
    0: 10016
    1: 10056
    1: 10056
Event f3:
    0: 10056
    0: 10016
    0: 10016
    1: 10056
    1: 10057
    1: 10016
Event f4:
    0: 10054
    0: 10054
    0: 10054
    0: 10054
    0: 10031
    0: 10031
    0: 10016
    1: 10054
    1: 10054
    1: 10031
    1: 10016
    1: 10016
    1: 10016
    1: 10016
    2: 10054
    2: 10049
    2: 10049
    2: 10049
    2: 10040
    2: 10016
    2: 10016
Event f5:
    0: 10054
    0: 10057
    0: 10065
    0: 10065
    0: 10023
    0: 10023
    0: 10023
    0: 10023
    0: 10023
    0: 10031
    0: 10016
    1: 10108
    1: 10054
    1: 10054
    1: 10057
    1: 10024
    1: 10065
    1: 10023
    1: 10012
    1: 10012
    1: 10016
    1: 10016
    2: 10054
    2: 10054
 

    1: 10030
    1: 10091
Event f56:
    0: 10053
    0: 10005
    0: 10049
    0: 10004
    1: 10030
    1: 10048
    1: 10005
    1: 10004
Event f57:
    0: 10031
    0: 10041
    0: 10002
    1: 10003
    1: 10003
    1: 10002
    2: 10026
    2: 10030
    2: 10053
Event f58:
    0: 10031
    0: 10004
    1: 10041
    1: 10002
Event f59:
    0: 10031
    0: 10004
    1: 10040
    1: 10012
Event f60:
    0: 10048
    0: 10042
    0: 10002
    1: 10030
    1: 10005
    1: 10005
    2: 10044
    2: 10044
    2: 10002
Event f61:
    0: 10030
    0: 10002
    0: 10002
    1: 10030
    1: 10048
    1: 10031
Event f62:
    0: 10005
    0: 10002
    0: 10004
    1: 10076
    1: 10030
    1: 10030
    2: 10044
    2: 10030
    2: 10002
Event f63:
    0: 10003
    0: 10042
    0: 10002
    1: 10048
    1: 10041
    1: 10002
Event f64:
    0: 10030
    0: 10042
    1: 10002
    1: 10002
Event f65:
    0: 10022
    0: 10006
    0: 10027
    1: 10002
    1: 10002
    1: 10002
Event f66:
    0: 1

    2: 10018
    2: 10018
    2: 10070
    2: 10040
Event f112:
    0: 10007
    0: 10042
    1: 10007
    1: 10014
Event f113:
    0: 10009
    0: 10002
    0: 10068
    1: 10017
    1: 10017
    1: 10040
Event f114:
    0: 10044
    0: 10044
    0: 10007
    0: 10033
    0: 10093
    0: 10080
    0: 10068
    0: 10021
    1: 10000
    1: 10000
    1: 10007
    1: 10009
    1: 10009
    1: 10010
    1: 10002
    1: 10002
    2: 10000
    2: 10044
    2: 10044
    2: 10009
    2: 10033
    2: 10039
    2: 10002
    2: 10014
Event f115:
    0: 10000
    0: 10000
    0: 10018
    0: 10002
    0: 10002
    1: 10044
    1: 10007
    1: 10010
    1: 10057
    1: 10080
Event f116:
    0: 10044
    0: 10007
    0: 10010
    0: 10010
    0: 10072
    0: 10075
    0: 10075
    0: 10015
    0: 10024
    0: 10005
    0: 10005
    0: 10005
    0: 10005
    0: 10005
    0: 10005
    0: 10062
    0: 10062
    0: 10073
    0: 10073
    0: 10038
    0: 10014
    1: 10000
    1: 10044
    1: 10005
    

    2: 10024
    2: 10099
    2: 10040
Event f158:
    0: 10022
    0: 10003
    0: 10003
    0: 10003
    0: 10003
    0: 10003
    0: 10006
    0: 10027
    0: 10040
    1: 10022
    1: 10022
    1: 10022
    1: 10022
    1: 10003
    1: 10003
    1: 10015
    1: 10040
    1: 10019
Event f159:
    0: 10043
    0: 10024
    0: 10040
    1: 10048
    1: 10024
    1: 10042
    2: 10043
    2: 10033
    2: 10014
Event f160:
    0: 10043
    0: 10018
    1: 10033
    1: 10024
Event f161:
    0: 10007
    0: 10003
    0: 10057
    0: 10057
    0: 10057
    0: 10024
    0: 10049
    0: 10092
    0: 10006
    0: 10040
    0: 10045
    0: 10045
    0: 10045
    1: 10022
    1: 10007
    1: 10007
    1: 10003
    1: 10056
    1: 10057
    1: 10024
    1: 10049
    1: 10040
    1: 10040
    1: 10045
    1: 10045
    1: 10045
    2: 10003
    2: 10003
    2: 10003
    2: 10003
    2: 10057
    2: 10057
    2: 10072
    2: 10049
    2: 10092
    2: 10012
    2: 10019
    2: 10019
    2: 10045
Eve

    2: 10024
    2: 10018
    2: 10028
    2: 10028
    2: 10028
    2: 10028
    2: 10052
    2: 10052
Event f203:
    0: 10059
    0: 10029
    0: 10024
    0: 10062
    0: 10028
    0: 10028
    0: 10045
    0: 10052
    1: 10024
    1: 10018
    1: 10028
    1: 10028
    1: 10028
    1: 10028
    1: 10052
    1: 10052
Event f204:
    0: 10029
    0: 10045
    1: 10029
    1: 10048
Event f205:
    0: 10000
    0: 10024
    0: 10028
    0: 10045
    0: 10052
    0: 10052
    1: 10024
    1: 10024
    1: 10062
    1: 10081
    1: 10028
    1: 10052
Event f206:
    0: 10024
    0: 10024
    1: 10029
    1: 10014
Event f207:
    0: 10053
    0: 10024
    0: 10024
    1: 10059
    1: 10024
    1: 10045
Event f208:
    0: 10053
    0: 10053
    0: 10024
    1: 10029
    1: 10029
    1: 10024
    2: 10059
    2: 10015
    2: 10024
Event f209:
    0: 10029
    0: 10024
    1: 10044
    1: 10024
Event f210:
    0: 10029
    0: 10024
    1: 10029
    1: 10052
Event f211:
    0: 10030
    0: 1

    2: 10029
    2: 10010
    2: 10080
    2: 10024
    2: 10074
    2: 10070
    2: 10011
    2: 10011
    2: 10011
Event f265:
Event f266:
    0: 10020
    0: 10029
    0: 10039
    0: 10060
    0: 10040
    0: 10011
    0: 10011
    1: 10010
    1: 10010
    1: 10010
    1: 10039
    1: 10039
    1: 10039
    1: 10038
    2: 10020
    2: 10087
    2: 10025
    2: 10010
    2: 10010
    2: 10010
    2: 10038
Event f267:
    0: 10029
    0: 10010
    0: 10010
    0: 10080
    0: 10015
    0: 10042
    0: 10040
    0: 10081
    0: 10011
    0: 10036
    0: 10036
    1: 10020
    1: 10087
    1: 10087
    1: 10003
    1: 10010
    1: 10015
    1: 10042
    1: 10038
    1: 10036
    1: 10036
    1: 10036
    2: 10029
    2: 10029
    2: 10029
    2: 10010
    2: 10010
    2: 10039
    2: 10027
    2: 10038
    2: 10036
    2: 10036
    2: 10036
Event f268:
    0: 10038
    0: 10011
    1: 10020
    1: 10015
Event f269:
    0: 10020
    0: 10009
    0: 10010
    0: 10010
    0: 10073
    

    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    0: 10019
    1: 10043
    1: 10043
    1: 10044
    1: 10044
    1: 10007
    1: 10007
    1: 10007
    1: 10007
    1: 10007
    1: 10007
    1: 10096
    1: 10030
    1: 10030
    1: 10030
    1: 10030
    1: 10048
    1: 10072
    1: 10107
    1: 10027
    1: 10014
    1: 10019
    1: 10019
    1: 10019
    2: 10043
    2: 10043
    2: 10044
    2: 10044
    2: 10044
    2: 10007
    2: 10007
    2: 10007
    2: 10007
    2: 10009
    2: 10033
    2: 10030
    2: 10030
    2: 10030
    2: 10075
    2: 10019
    2: 10019
    2: 10019
    2: 10019
    2: 10019
    2: 10019
    2: 10086
    2: 10086
Event s1:
    0: 10016
    1: 10065
Event s2:
    0: 10040
    1: 10016
    2: 10065
Event s3:
Event s4:
Event s5:
    0: 10016
    1: 10065
Event s6:
    0: 10056
    1: 10016
Event s7:
    0: 10016
    1: 10057
Event s8:
    0: 10016
    1: 10057
Event s9:
Event s10:
  

Event s249:
    0: 10005
    1: 10000
Event s250:
    0: 10005
    1: 10062
Event s251:
    0: 10018
    1: 10042
Event s252:
    0: 10048
    1: 10015
Event s253:
    0: 10014
    1: 10018
Event s254:
Event s255:
    0: 10018
    1: 10014
Event s256:
    0: 10018
    1: 10053
Event s257:
    0: 10048
    1: 10042
Event s258:
    0: 10014
    1: 10091
Event s259:
    0: 10048
    1: 10018
Event s260:
    0: 10042
    1: 10018
Event s261:
    0: 10042
    1: 10019
Event s262:
Event s263:
Event s264:
Event s265:
    0: 10018
    1: 10003
Event s266:
Event s267:
    0: 10048
    1: 10015
Event s268:
    0: 10003
    1: 10000
    2: 10022
Event s269:
    0: 10042
    1: 10017
Event s270:
Event s271:
Event s272:
Event s273:
    0: 10019
    1: 10018
Event s274:
    0: 10018
    1: 10014
Event s275:
    0: 10048
    1: 10022
Event s276:
    0: 10022
    1: 10018
Event s277:
Event s278:
    0: 10033
    1: 10048
Event s279:
    0: 10033
    1: 10018
Event s280:
    0: 10012
    1: 10014
Event

In [139]:
events['s310']

{'Category': 'Sim',
 'Type': 'ISS',
 'CrewType': None,
 'CrewRequirements': ['', ''],
 'CrewMin': 2,
 'CrewMax': 2,
 'StartDay': 174.0,
 'EndDay': 174.0}

In [149]:
solution = solutionTemplate()
for event in events:
    for role in range(len(events[event]['CrewRequirements'])):
        for pilot in pilots:
            if pulp.value(assignments[event, pilot, events[event]['CrewRequirements'][role]]) == 1:
                solution['Staff'][pilot].append(event)
                
checkSchedule(solution, verbose=True)

# Big Rocks
# 1. need to make pilots unavailable once they start a mission
# 2. if crew requirements are empty string, algorithm doesn't assign pilots


Info: checking schedule passed in as dictionary.
Incorrect Solution Error: 10000 assigned to overlapping events: f106 and f109.
Incorrect Solution Error: 10000 assigned to overlapping events: f109 and f111.
Incorrect Solution Error: 10000 assigned to overlapping events: f111 and Unavailable.
Incorrect Solution Error: 10000 assigned to overlapping events: Unavailable and f118.
Incorrect Solution Error: 10000 assigned to overlapping events: f118 and s193.
Incorrect Solution Error: 10000 assigned to overlapping events: s193 and s195.
Incorrect Solution Error: 10000 assigned to overlapping events: f123 and f124.
Incorrect Solution Error: 10000 assigned to overlapping events: f124 and s202.
Incorrect Solution Error: 10000 assigned to overlapping events: f141 and f142.
Incorrect Solution Error: 10000 assigned to overlapping events: f142 and Unavailable.
Incorrect Solution Error: 10000 assigned to overlapping events: f145 and f146.
Incorrect Solution Error: 10000 assigned to overlapping event

False